# Generación de Texto con LSTM usando 'Cien Años de Soledad'

En este cuaderno, vamos a implementar un modelo LSTM que será entrenado usando el texto del libro 'Cien Años de Soledad' de Gabriel García Márquez.
El objetivo es que el modelo aprenda el estilo literario y sea capaz de generar texto similar al del autor.

## Requisitos previos
- Python 3.7+
- TensorFlow
- Numpy
- Matplotlib

## Objetivo
1. Preprocesar el texto de 'Cien Años de Soledad'.
2. Implementar un modelo LSTM para la generación de texto.
3. Entrenar el modelo y generar texto nuevo.

## 1. Cargando y Preprocesando el Texto
Cargaremos el texto de 'Cien Años de Soledad' y lo preprocesaremos para convertirlo en secuencias de texto adecuadas para el entrenamiento del modelo LSTM.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import requests
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


# Cargar el texto de 'Cien Años de Soledad'
url = 'https://raw.githubusercontent.com/Izainea/nlp_ean/main/Datos/Datos%20Crudos/CAS.txt'
response = requests.get(url)

if response.status_code == 200:
    text = response.text[:30000]
else:
    print('Error al descargar el archivo.')

#Creamos las secuencias usando el tokenizador de nltk

tokens=nltk.word_tokenize(text)

n=10

input_sequences=[]

for i in range(n,len(tokens)):
    n_gram=tokens[i-n:i]
    input_sequences.append(n_gram)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
corpus= [' '.join(n_gram) for n_gram in input_sequences]

# Tokenizar el texto
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(tokenizer.word_index[','])

# Convertir texto en secuencias de palabras
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Padding para igualar las secuencias
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Dividir en características y etiquetas
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

X.shape, y.shape

2


((50040, 9), (50040, 1822))

## 2. Creando el Modelo LSTM
Ahora crearemos el modelo LSTM que será entrenado para predecir la siguiente palabra en una secuencia, basado en el estilo literario del libro.

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN


# Crear el modelo RNN
model_rnn = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=8, input_length=max_sequence_len-1),
    SimpleRNN(32),
    Dense(len(tokenizer.word_index)+1, activation='softmax')
])



# Compilar el modelo
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Resumen del modelo
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 9, 8)              14576     
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1312      
                                                                 
 dense (Dense)               (None, 1822)              60126     
                                                                 
Total params: 76014 (296.93 KB)
Trainable params: 76014 (296.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 3. Entrenando el Modelo
Entrenaremos el modelo durante 100 épocas para que aprenda las secuencias de texto y las relaciones entre palabras.

In [4]:
# Entrenar el modelo
history = model_rnn.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
1564/1564 [==============================] - 8s 4ms/step - loss: 5.6649 - accuracy: 0.0809
Epoch 2/10
1564/1564 [==============================] - 7s 4ms/step - loss: 4.9988 - accuracy: 0.1119
Epoch 3/10
1564/1564 [==============================] - 7s 4ms/step - loss: 4.6064 - accuracy: 0.1426
Epoch 4/10
1564/1564 [==============================] - 7s 4ms/step - loss: 4.2487 - accuracy: 0.1727
Epoch 5/10
1564/1564 [==============================] - 7s 4ms/step - loss: 3.9128 - accuracy: 0.2106
Epoch 6/10
1564/1564 [==============================] - 7s 4ms/step - loss: 3.5956 - accuracy: 0.2576
Epoch 7/10
1564/1564 [==============================] - 7s 4ms/step - loss: 3.2924 - accuracy: 0.3102
Epoch 8/10
1564/1564 [==============================] - 7s 4ms/step - loss: 3.0164 - accuracy: 0.3598
Epoch 9/10
1564/1564 [==============================] - 7s 4ms/step - loss: 2.7699 - accuracy: 0.4084
Epoch 10/10
1564/1564 [==============================] - 7s 4ms/step - loss: 2.557

## 4. Generación de Texto
Una vez que el modelo esté entrenado, podemos usarlo para generar texto nuevo en el estilo de 'Cien Años de Soledad'.

In [7]:
def generar_texto(model, tokenizer, seed_text, max_sequence_len, n_words):
    for _ in range(n_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generar texto
print(generar_texto(model_rnn, tokenizer, 'muchos años después', max_sequence_len, 50))

1/1 [==============================] - 0s 22ms/step
muchos años después , no le permitía derribar un tibio olor de sangre . josé arcadio buendía , que aún no acababa de agua por tratar de cerdo a macondo , y el cuartito del galeón , y allí penetraron al bosque por un enorme calabazo lleno de melquíades . « se pronto


In [8]:
# Entrenar el modelo
history = model_rnn.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
1564/1564 [==============================] - 7s 4ms/step - loss: 2.3752 - accuracy: 0.4822
Epoch 2/20
1564/1564 [==============================] - 7s 5ms/step - loss: 2.2202 - accuracy: 0.5113
Epoch 3/20
1564/1564 [==============================] - 7s 4ms/step - loss: 2.0851 - accuracy: 0.5404
Epoch 4/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.9705 - accuracy: 0.5626
Epoch 5/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.8716 - accuracy: 0.5828
Epoch 6/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.7855 - accuracy: 0.5992
Epoch 7/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.7092 - accuracy: 0.6161
Epoch 8/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.6434 - accuracy: 0.6294
Epoch 9/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.5827 - accuracy: 0.6422
Epoch 10/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.529

In [9]:
print(generar_texto(model_rnn, tokenizer, 'muchos años después', max_sequence_len, 50))

1/1 [==============================] - 0s 22ms/step
muchos años después , el coronel aureliano buendía , la única posibilidad de contacto con la civilización era la puerta del cuartito , y un chaleco de terciopelo patinado por el verdín de los pájaros . de modo que dotó de herramientas de desmonte y armas de cacería a los mismos hombres que


In [ ]:
# Entrenar el modelo
history = model_rnn.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
1564/1564 [==============================] - 7s 5ms/step - loss: 1.1624 - accuracy: 0.7378
Epoch 2/20
1564/1564 [==============================] - 7s 5ms/step - loss: 1.1417 - accuracy: 0.7422
Epoch 3/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.1218 - accuracy: 0.7471
Epoch 4/20
1564/1564 [==============================] - 7s 5ms/step - loss: 1.1033 - accuracy: 0.7524
Epoch 5/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.0852 - accuracy: 0.7547
Epoch 6/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.0682 - accuracy: 0.7573
Epoch 7/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.0535 - accuracy: 0.7626
Epoch 8/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.0383 - accuracy: 0.7655
Epoch 9/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.0244 - accuracy: 0.7698
Epoch 10/20
1564/1564 [==============================] - 7s 4ms/step - loss: 1.010

In [ ]:
print(generar_texto(model_rnn, tokenizer, 'muchos años después', max_sequence_len, 50))